In [26]:
#this is the initial import of the diff files, loading them into pandas dataframes
import pandas as pd
import os
import numpy as np
from csv_diff import load_csv, compare
folderpath = r"D:\jayData\monthly_human_diffs" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
all_files = []
print(filepaths)
header = ['tax_id', 'GeneID', 'Symbol', 'LocusTag', 'Synonyms', 'dbXrefs', 'chromosome', 'map_location', 'description', 'type_of_gene', 'Symbol_from_nomenclature_authority', 'Full_name_from_nomenclature_authority', 'Nomenclature_status', 'Other_designations', 'Modification_date', 'Feature_type']
dfList = []
# data = pd.read_csv('D:\\jayData\\yearly_diffs\\2008_2009_diff.csv', delimiter = '\t', skiprows = 5, error_bad_lines = False)
# data.columns = header
for i in range(6):
    globals()['df%s' % i] = pd.read_csv(filepaths[i], delimiter = "	", header = None, dtype = str)
    globals()['df%s' % i].columns = header
    dfList.append(globals()['df%s' % i])

['D:\\jayData\\monthly_human_diffs\\human_20200422_20200710.diff', 'D:\\jayData\\monthly_human_diffs\\human_20200710_20200810.diff', 'D:\\jayData\\monthly_human_diffs\\human_20200810_20201031.diff', 'D:\\jayData\\monthly_human_diffs\\human_20201031_20201129.diff', 'D:\\jayData\\monthly_human_diffs\\human_20201129_20201231.diff', 'D:\\jayData\\monthly_human_diffs\\human_20201231_20210109.diff']


In [29]:
print(dfList[1])

       tax_id     GeneID           Symbol LocusTag                   Synonyms  \
0       -9606          1             A1BG        -       A1B|ABG|GAB|HYST2477   
1       -9606          2              A2M        -  A2MD|CPAMD5|FWP007|S863-7   
2       -9606          3            A2MP1        -                       A2MP   
3       -9606          9             NAT1        -       AAC1|MNAT|NAT-1|NATI   
4       -9606         10             NAT2        -            AAC2|NAT-2|PNAT   
...       ...        ...              ...      ...                        ...   
120090   9606  118027463             SRS2        -                          -   
120091   9606  118097967          ELOCP26        -            ELOC26|TCEB1P26   
120092   9606  118126072           GUSBP9        -                          -   
120093  -9606  118142757  GUCA1ANB-GUCA1A        -                          -   
120094  +9606  118142757           GUCA1A        -            GUCA1ANB-GUCA1A   

                           

In [ ]:
newDfList = []
n = 0

for df in dfList:
    print(n)
    diff_symbols = []
    df = df[['tax_id', 'GeneID', 'Synonyms', 'dbXrefs', 'chromosome', 'Modification_date']]
#     df = df[df['tax_id'].str.contains('9606', na = False)]
    for i, row in df.iterrows():
        diff_symbols.append(df.at[i, 'tax_id'][0])
#         if df.at[i, 'tax_id'][0] == ' ':
#             df.drop(i, inplace = True)

    synonym_count = []
    dbXrefs_count = []
    has_hgnc = []
    has_mim = []
    has_ensembl = []
    print('syns')##############################################################
    for i, row in df.iterrows():
        if isinstance(df.at[i, 'Synonyms'], str):
            if df.at[i, 'Synonyms'] == '-':
                synonym_count.append(0)
            else:
                synonym_count.append(str(df.at[i, 'Synonyms'].count('|') + 1))
        if isinstance(df.at[i, 'Synonyms'], float):
            synonym_count.append(0)
    print('dbXrefs')###########################################################
    for i, row in df.iterrows():
        val = df.at[i, 'dbXrefs']
        if isinstance(val, str):
            if val == '-':
                dbXrefs_count.append(0)
                has_hgnc.append(False)
                has_mim.append(False)
                has_ensembl.append(False)
            else:
                dbXrefs_count.append(str(val.count('|') + 1))
                has_hgnc.append('HGNC' in val)
                has_mim.append('MIM' in val)
                has_ensembl.append('Ensembl' in val)
        if isinstance(val, float):
            dbXrefs_count.append(0)
            has_hgnc.append(False)
            has_mim.append(False)
            has_ensembl.append(False)
    print('indels')############################################################
    indelupdateCols = []
    for i, row in df.iterrows():
#         temp = df[df['tax_id'].str.contains(str(df.at[i, 'GeneID']), na = False)]
        sub = df[df.values == df.at[i, 'GeneID']]
        print(df.at[i, 'GeneID'])
        if sub.shape[0] > 1:
            if sub.iloc[0]['Modification_date'] != sub.iloc[1]['Modification_date']:
                indelupdateCols.append('update - date only')
            else:
                indelupdateCols.append('update')
        if sub.shape[0] <= 1:
            if df.at[i, 'tax_id'][0] == ' ':
                indelupdateCols.append('NA')
            if df.at[i, 'tax_id'][0] == '+':
                indelupdateCols.append('insertion')
            if df.at[i, 'tax_id'][0] == '-':
                indelupdateCols.append('deletion')
            if df.at[i, 'tax_id'][0] == '@':
                indelupdateCols.append('NA')
                
    for i, row in df.iterrows():
        df.at[i, 'tax_id'] = df.at[i,'tax_id'][1:]
        
    df['synonym_count'] = synonym_count
    df['dbXrefs_count'] = dbXrefs_count
    df['has_hgnc'] = has_hgnc
    df['has_mim'] = has_mim
    df['has_ensembl'] = has_ensembl
    df['diff_symbol'] = diff_symbols
    df['insert_delete_or_update'] = indelupdateCols
    newDfList.append(df.copy())
    df.to_csv('d:/jayData/2020_12_human_diff_analyses/monthlyDiffAnalysis%s.csv' % n, index = False, sep = '\t')
    n += 1
    